In [1]:
import numpy as np
from pressure_vessel_edit import PressureVessel
import random 
import time
import statistics

In [2]:
simulated_index=[]
batch_start=11
filename= 'data_'+str(batch_start)+'.csv'
rhog_safety=15096.9      # calculted as pho=1027, g=9.8, safety factor=1.5 ( from STR excel sheet)
threshold=0.05           # 5 percent ( Coefficient of variation= std/mean) 
window_len= 6            # window to check convergence

In [15]:
designset_load= np.loadtxt('./indexed_design_point.txt', delimiter = " ")
design_set=designset_load[batch_start:batch_start+1,:]
design_set

array([[ 12.  , 100.  ,   0.1 ,   0.12]])

In [16]:
vessel = PressureVessel('pv_capsule1.FCStd')
vessel.set_tensile_strength(276)
vessel.set_youngs_modulus(68900)
csvwriter=vessel.csv_open_output(filename)
vessel.print_info() 

Opening: pv_capsule1.FCStd
file name is: data_11.csv
opening file to write
field name is: ['thickness', 'radius', 'length']
Object names: Body, Origin, X_Axis, Y_Axis, Z_Axis, XY_Plane, XZ_Plane, YZ_Plane, Sketch, Revolution, Analysis, SolverCcxTools, MaterialSolid, FEMMeshGmsh, ConstraintFixed, ConstraintPressure
Sketch parameters:
thickness = 0.01
radius = 0.3371301208345706
length = 0.1
Body properties:
  body_area = 3.190399 m^2
  body_volume = 0.015949898 m^3
  body_mass = 43.065 kg
  outer_area = 1.640078 m^2
  outer_volume = 0.196208618 m^3
  inner_area = 1.550321 m^2
  inner_volume = 0.180258720 m^3
FEM parameters:
  pressure = 3.01938 MPa
  mesh_length = 0.054000000000000006 m
Material parameters:
  youngs_modulus = 68900.0 MPa
  poisson_ratio = 0.33
  tensile_strength = 276.0 MPa
  density = 2700.0 kg/m^3
Mesh properties:
  node_count = 8395
  edge_count = 194
  face_count = 2786
  volume_count = 4210
FEM Results: none


In [17]:
convergence_data=[]

def listOfTuples(l1, l2):
    return list(map(lambda x, y:(x,y), l1, l2))

def capture_datumerror(crush_pressure,thickness_d,radius_d,length_d):
    try: 
     vessel.set_pressure(crush_pressure)
     vessel.set('thickness', thickness_d)
     vessel.set('radius', radius_d)
     vessel.set('length', length_d)
     return 0
    except:
     print("Something went wrong in setting value")
     return 1

def check_convergence(predictions,ml,experiment_id):
    if len(predictions)<=window_len :
        return False
    else: 
       windowed_prediction = predictions[-window_len:]
       windowed_ml = ml[-window_len:]
       print('windowed result is:',windowed_prediction)
       std_w=statistics.stdev(windowed_prediction)
       mean_w=statistics.mean(windowed_prediction)
       if (std_w/mean_w)>threshold:
          return False
       else:
          convergence_data.append(experiment_id)
          convergence_data.append(listOfTuples(windowed_prediction,windowed_ml))
          return True

In [18]:
skipped=[]
start=time.time()
#mesh_options=[0.08,0.04,0.035,0.030,0.025,0.020,0.01,0.008,0.004,0.002,0.001]
mesh_options=np.arange(.002,0.1,.002)[::-1]
for i in range(design_set.shape[0]):
     experiment_id=design_set[i][0]
     depth= design_set[i][1]
     length_d= design_set[i][2]
     thickness_d= design_set[i][3]
     radius_d= thickness_d+ np.random.rand()*0.4+0.1  
      
     print('---> depth is:',depth,'length is:',length_d,'thickness is:',thickness_d, 'radius is:',radius_d )
    
        
     crush_pressure= rhog_safety*depth*0.000001
   
     de=capture_datumerror(crush_pressure,thickness_d,radius_d,length_d)
     vessel.set_exp_index(experiment_id)
     predicted_vonmises=[]; mesh_lengths=[]
     if de==0:
      k=0 
      for j in range(len(mesh_options)):
        ml= mesh_options[j]
        print('ml is:',ml)
        vessel.set('mesh_length', ml)
        k=vessel.run_analysis()
        print('itr is:',i,',k is:',k, ',ml is:',ml)
        if k==2:  
            mesh_lengths.append(ml)
            predicted_vonmises.append(vessel.get_vonmises_stress())
            print('predicted vonmises is:',predicted_vonmises)
            convergence=check_convergence(predicted_vonmises,mesh_lengths,experiment_id)
            if convergence==True: 
              break  
      if k==2 :
       vessel.csv_write_row(csvwriter)
       simulated_index.append(batch_start+i)
      else:
       skipped.append(batch_start+i)
     else: 
       skipped.append(batch_start+i)
     with open ("convergence.txt","w")as fp: 
       fp.write(str(convergence_data))
vessel.csv_close_output(csvwriter)



---> depth is: 100.0 length is: 0.1 thickness is: 0.12 radius is: 0.3018885453562702
ml is: 0.098
Running GMSH mesher ... 1756 nodes, 84 edges, 588 faces, 868 volumes
Running FEM analysis ... vonMises stress: 2.86 MPa
itr is: 0 ,k is: 2 ,ml is: 0.098
predicted vonmises is: [2.861710087297104]
ml is: 0.096
Running GMSH mesher ... 1760 nodes, 84 edges, 590 faces, 869 volumes
Running FEM analysis ... vonMises stress: 2.86 MPa
itr is: 0 ,k is: 2 ,ml is: 0.096
predicted vonmises is: [2.861710087297104, 2.861885051665423]
ml is: 0.094
Running GMSH mesher ... 2180 nodes, 92 edges, 728 faces, 1082 volumes
Running FEM analysis ... vonMises stress: 2.88 MPa
itr is: 0 ,k is: 2 ,ml is: 0.094
predicted vonmises is: [2.861710087297104, 2.861885051665423, 2.876118664692227]
ml is: 0.092
Running GMSH mesher ... 2180 nodes, 92 edges, 728 faces, 1082 volumes
Running FEM analysis ... vonMises stress: 2.88 MPa
itr is: 0 ,k is: 2 ,ml is: 0.092
predicted vonmises is: [2.861710087297104, 2.861885051665423, 2